# CS246 - Colab 1
## Wordcount in Spark

### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 281.3 MB 39 kB/s 
     |████████████████████████████████| 198 kB 71.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=db787d003fc20727b7c8d044dcaeded02b868123b3abb796ea555d29f6dfc0b0
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Rea

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
id='1SE6k_0YukzGd5wK-E4i6mG83nydlfvSa'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('pg100.txt')

If you executed the cells above, you should be able to see the file *pg100.txt* under the "Files" tab on the left panel.

### Your task

If you run successfully the setup stage, you are ready to work on the *pg100.txt* file which contains a copy of the complete works of Shakespeare.

Write a Spark application which outputs the number of words that start with each letter. This means that for every letter we want to count the total number of (non-unique) words that start with a specific letter. In your implementation **ignore the letter case**, i.e., consider all words as lower case. Also, you can ignore all the words **starting** with a non-alphabetic character.

In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [6]:
# YOUR
txt = spark.read.text('pg100.txt')
txt.show(10)

+--------------------+
|               value|
+--------------------+
|The Project Guten...|
| William Shakespeare|
|                    |
|This eBook is for...|
|almost no restric...|
|re-use it under t...|
|with this eBook o...|
|                    |
|** This is a COPY...|
|**     Please fol...|
+--------------------+
only showing top 10 rows



In [8]:
txt.take(2)

[Row(value='The Project Gutenberg EBook of The Complete Works of William Shakespeare, by'),
 Row(value='William Shakespeare')]

From above, we can know that the `spark.read.text` read text line by line.

More: https://spark.apache.org/docs/latest/sql-data-sources-text.html

There is no option to read word by word.

## Start process the data

keep a table in every machine with alphabet `[A-Z]`, and keep count in each machine. Finally, just merge the data and get the final answer.

In [22]:
start_alphabets = txt.rdd.map(lambda row: [a[0].lower() for a in row.value.split()])
start_alphabets.take(3)

[['t', 'p', 'g', 'e', 'o', 't', 'c', 'w', 'o', 'w', 's', 'b'], ['w', 's'], []]

In [36]:
import numpy as np
def alphabet2dict(row):
  counter = [0]*26
  for a in [a[0].lower() for a in row.value.split()]:
    pos = ord(a) - ord('a')
    if pos >= 0 and pos < 26:
      counter[pos] += 1
  return np.array(counter)

counted_alphabets = txt.rdd.map(lambda row: alphabet2dict(row))
counted_alphabets.take(3)

[array([0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 1, 2, 0, 0,
        2, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0])]

In [40]:
alphabet_counter = counted_alphabets.reduce(lambda a, b: a + b)
alphabet_counter

array([ 84836,  45455,  34567,  29713,  18697,  36814,  20782,  60563,
        62167,   3339,   9418,  29569,  55676,  26759,  43494,  27759,
         2377,  14265,  65705, 123602,   9170,   5728,  59597,     14,
        25855,     71])

In [44]:
chr(ord('a')+1)

'b'

In [45]:
for i in range(26):
  a = chr(i+ord('a'))
  print(f'{a}: {alphabet_counter[i]}')

a: 84836
b: 45455
c: 34567
d: 29713
e: 18697
f: 36814
g: 20782
h: 60563
i: 62167
j: 3339
k: 9418
l: 29569
m: 55676
n: 26759
o: 43494
p: 27759
q: 2377
r: 14265
s: 65705
t: 123602
u: 9170
v: 5728
w: 59597
x: 14
y: 25855
z: 71


Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!